### Importing the Libraries

In [52]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os

In [53]:
tf.__version__ 

'2.8.0'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

In [54]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [55]:
test_datagen = ImageDataGenerator(rescale = 1./255)

### Creating the Training set

In [56]:
# path to training set 
trainingDir = '../dataSet/trainAlgoData'
training_set = train_datagen.flow_from_directory(trainingDir,                                
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 225 images belonging to 2 classes.


In [57]:
test_set = test_datagen.flow_from_directory('../dataSet/testAlgoData',
                                            target_size = (128, 128),                                  
                                            batch_size = 3,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 15 images belonging to 2 classes.


### Part 2 - Building the CNN

#### Initializing the CNN

In [58]:
classifier = tf.keras.models.Sequential()

#### Step 1 - Convolution

In [59]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

#### Step 2 - Pooling

In [60]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Adding a second convolutional layer

In [61]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Step 3 - Flattening

In [62]:
classifier.add(tf.keras.layers.Flatten())

#### Step 4 - Full Connection

In [63]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=2, activation='softmax')) # softmax for more than 27

### Part 3 - Training the CNN

#### Compiling the CNN

In [64]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [65]:
classifier.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 32768)             0         
                                                                 
 dense_16 (Dense)            (None, 128)              

In [66]:
classifier.fit(training_set,
                  epochs = 10,
                  validation_data = test_set)

Epoch 1/10
23/23 [==============================] - 1s 43ms/step - loss: 0.7376 - accuracy: 0.5556 - val_loss: 0.6364 - val_accuracy: 0.2667
Epoch 2/10
23/23 [==============================] - 1s 40ms/step - loss: 0.3589 - accuracy: 0.8978 - val_loss: 0.0045 - val_accuracy: 1.0000
Epoch 3/10
23/23 [==============================] - 1s 40ms/step - loss: 0.1059 - accuracy: 0.9733 - val_loss: 1.4148e-04 - val_accuracy: 1.0000
Epoch 4/10
23/23 [==============================] - 1s 40ms/step - loss: 0.0741 - accuracy: 0.9822 - val_loss: 2.8184e-04 - val_accuracy: 1.0000
Epoch 5/10
23/23 [==============================] - 1s 39ms/step - loss: 0.0698 - accuracy: 0.9822 - val_loss: 7.3348e-05 - val_accuracy: 1.0000
Epoch 6/10
23/23 [==============================] - 1s 40ms/step - loss: 0.0692 - accuracy: 0.9956 - val_loss: 6.2329e-04 - val_accuracy: 1.0000
Epoch 7/10
23/23 [==============================] - 1s 39ms/step - loss: 0.0325 - accuracy: 0.9911 - val_loss: 1.0734e-04 - val_accuracy: 

#### Saving the Model

In [67]:
model_json = classifier.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_new.h5')
print('Weights saved')

Model Saved
Weights saved
